In [4]:
import torch
import torch.nn.functional as F

# 定义前馈神经网络模型
class FeedforwardNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # 初始化权重和偏置
        self.W1 = torch.randn(input_size, hidden_size, requires_grad=True)
        self.b1 = torch.zeros(hidden_size, requires_grad=True)
        self.W2 = torch.randn(hidden_size, output_size, requires_grad=True)
        self.b2 = torch.zeros(output_size, requires_grad=True)

    def forward(self, x):
        # 第一个隐藏层
        h1 = torch.relu(torch.matmul(x, self.W1) + self.b1)
        # 输出层
        out = torch.matmul(h1, self.W2) + self.b2
        return out

# 定义均方误差损失函数
def mean_squared_error(y_pred, y_true):
    return ((y_pred - y_true)**2).mean()

# 设置参数和数据
num_inputs = 500
n_train = 7000
n_test = 3000
true_w, true_b = torch.ones(num_inputs, 1) * 0.01, 0.05
features = torch.randn((n_train + n_test, num_inputs))
labels = torch.matmul(features, true_w) + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)
train_features, test_features = features[:n_train, :], features[n_train:, :]
train_labels, test_labels = labels[:n_train], labels[n_train:]

# 初始化模型参数
hidden_size = 128
output_size = 1
model = FeedforwardNN(num_inputs, hidden_size, output_size)

# 训练模型
num_epochs = 100
batch_size = 100
lr = 0.001



In [7]:
train_labels

In [5]:
for epoch in range(num_epochs):
    for i in range(0, n_train, batch_size):
        X = train_features[i:i + batch_size]
        y = train_labels[i:i + batch_size]

        # 前向传播
        y_pred = model.forward(X)
        loss = mean_squared_error(y_pred, y)

        # 反向传播
        loss.backward()

        # 使用随机梯度下降优化算法更新参数
        with torch.no_grad():
            model.W1 -= lr * model.W1.grad / batch_size
            model.b1 -= lr * model.b1.grad / batch_size
            model.W2 -= lr * model.W2.grad / batch_size
            model.b2 -= lr * model.b2.grad / batch_size

            # 清除梯度
            model.W1.grad.zero_()
            model.b1.grad.zero_()
            model.W2.grad.zero_()
            model.b2.grad.zero_()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 测试模型
with torch.no_grad():
    test_outputs = model.forward(test_features)
    test_loss = mean_squared_error(test_outputs, test_labels)
    print(f"Test Loss: {test_loss.item():.4f}")
